In [1]:
import os
import torch
import torchani
import numpy as np

from pathlib import Path
from ase.units import Bohr, Hartree, eV, kcal, mol
from ase.io import read as aseread


In [2]:
def compute_ani_energy_and_forces(Z_path, R_path, device):
    # Load your numpy arrays
    Z = np.load(Z_path)  # Atomic numbers
    R = np.load(R_path)  # Positions

    # Convert numpy arrays to torch tensors
    Z = torch.tensor(Z, dtype=torch.long)
    R = torch.tensor(R, dtype=torch.float, requires_grad=True)

    # Load pretrained ANI-1x model
    model = torchani.models.ANI1x(periodic_table_index=True).to(device)

    # Forward pass to compute energy
    energy = model((Z, R)).energies
    derivative = torch.autograd.grad(energy.sum(), R)[0]
    force = -derivative

    return energy, force


In [16]:
pwd

'/vast/projects/ml4chem/NikitaFedik/GITHUB/ml_tps_si'

In [17]:
ls data/AD/thermal_MD_10k

ad_E_hippnts_kcal_mol.npy         ad_F_QM_kcal_mol_A.npy  ad_Z.npy
ad_E_QM_kcal_mol.npy              ad_paths.npy            DFT-logs/
ad_forces_hippnts_kcal_mol_A.npy  ad_R.npy


In [18]:
data_path = Path("data/AD/thermal_MD_10k")
# Example usagez
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
energy, force = compute_ani_energy_and_forces(data_path / 'ad_Z.npy', 
                                              data_path / 'ad_R.npy', device) 
# forces already have correct sign - see function above

In [19]:
F_kcal = force * Hartree / (kcal / mol) # to kcal/mol/A using ASE units 
E_kcal = energy * Hartree / (kcal / mol)  # to kcal/mol using ASE units 
E_kcal_shifted = E_kcal - torch.mean(E_kcal) # substract mean energy to get relative formation E

In [20]:
E_kcal_shifted


tensor([-7.9467,  4.5633,  9.0879,  ..., -0.6979,  3.8325,  2.4054],
       dtype=torch.float64, grad_fn=<SubBackward0>)

In [21]:
np.save('data/AD/thermal_MD_10k/E_ani_kcal_mol.npy', E_kcal_shifted.detach().numpy())
np.save('data/AD/thermal_MD_10k/forces_ani_kcal_mol_A.npy', F_kcal.detach().numpy())

#### Azobenzene

In [22]:
az_data_path = Path("data/AZ/thermal_MD_10k")
# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
az_energy, az_force = compute_ani_energy_and_forces(az_data_path / 'az_Z.npy', 
                                          az_data_path / 'az_R.npy', device)

In [23]:
az_E_kcal = az_energy * Hartree / (kcal / mol)  # to kcal/mol using ASE units 
az_F_kcal = az_force * Hartree / (kcal / mol) # to kcal/mol/A using ASE units 
az_E_kcal_shifted = az_E_kcal - torch.mean(az_E_kcal) # substract mean energy to get relative formation E
np.save('data/AZ/thermal_MD_10k/az_E_ani_kcal_mol.npy', az_E_kcal_shifted.detach().numpy())
np.save('data/AZ/thermal_MD_10k/az_F_ani_kcal_mol_A.npy', az_F_kcal.detach().numpy())

### Azobenzene - open shell rotation path

In [ ]:
xyz_path = Path('data/AZ/os-rotation/AZ_os-DFT_rotation_path.xyz')
atoms_list = aseread(xyz_path, index=":")

In [28]:
# get all atomic position in shape N mol, N atoms, 3
R = np.array([atoms.get_positions() for atoms in atoms_list])                                  

In [29]:
# we can use structure to get the atomic numbers
Z = atoms_list[0].get_atomic_numbers()
Z = torch.Tensor(np.tile(Z, (R.shape[0], 1))) # repeat Z tensor

In [30]:
Z.shape

torch.Size([270, 24])

In [31]:
R.shape

(270, 24, 3)

In [33]:
Z = torch.tensor(Z, dtype=torch.long)
R = torch.tensor(R, dtype=torch.float, requires_grad=True)

/vast/home/fns/.conda/envs/qm/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Forward pass to compute energy
model = torchani.models.ANI1x(periodic_table_index=True).to(device)
os_iso_E = model((Z, R)).energies
os_iso_E_kcal = os_iso_E * Hartree / (kcal / mol)  # to kcal/mol using ASE units


In [35]:
os_iso_E_kcal_shifted = os_iso_E_kcal - os_iso_E_kcal[0]

In [36]:
np.save('data/AZ/os-rotation/os_iso_E_torchani_kcal_mol.npy', os_iso_E_kcal_shifted.detach().numpy())

In [38]:
### Azobenzene -  closed shell inversion path

In [44]:
cd_xyz_path = Path('data/AZ/cs-inversion/AZ_cs-DFT_inversion_path.xyz')
cs_atoms_list = aseread(cd_xyz_path, index=":")
    
cs_Z = cs_atoms_list[0].get_atomic_numbers()
cs_Z = torch.Tensor(np.tile(cs_Z, (cs_R.shape[0], 1))) # repeat cs_Z tensor
cs_Z = torch.tensor(cs_Z, dtype=torch.long)

cs_R = np.array([atoms.get_positions() for atoms in cs_atoms_list])  
cs_R = torch.tensor(cs_R, dtype=torch.float, requires_grad=True)

/vast/home/fns/.conda/envs/qm/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [49]:
cs_iso_E = model((cs_Z, cs_R)).energies
cs_iso_E_kcal = cs_iso_E * Hartree / (kcal / mol)  # to kcal/mol using ASE units
cs_iso_E_kcal_shifted = cs_iso_E_kcal - cs_iso_E_kcal[0]

In [51]:
np.save('data/AZ/cs-inversion/cs_inv_E_torchani_kcal_mol.npy', cs_iso_E_kcal_shifted.detach().numpy())